# LLM agent Hackathon 
## Alireza Ghafarollahi, MIT

# AG2: Building LLM-powered multi-agent systems

In [ ]:
!pip install ag2
!pip install openai

In [19]:
import json
import os

with open('config.json') as f:
    config = json.load(f)

OPENAI_API_KEY = config["api_key"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [20]:
import autogen
autogen.__version__

'0.9.1post0'

# Example: Group chat for an online shopping problem

## Let's add a human customer

- ## Product Finder

- ## Purchase Advisor

- ## Checkout Assistant

- ## Customer (human)

## Creating the Necessary Agents

In [15]:
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern

llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

# Product finder agent
finder_message = """You are a smart product finder agent.
Given a general need or interest, suggest a suitable product someone might want to buy online. Mention the product name and a one-line reason why it's a good fit.
"""

# Purchase advisor agent
advisor_message = """You are a purchase advisor agent.
Given a product name, generate pros and cons, price estimate, and suggest one alternative product with reasoning.
"""

# Checkout assistant agent
checkout_message = """You are a checkout assistant agent.
Given a chosen product, provide a brief checkout summary including product, estimated price, shipping method, and a friendly thank-you message.
"""

# Customer initiates the shopping process
customer_message = """You are an online shopper.
You describe your needs, review purchase advice (with one round of follow-up), and then confirm a product to buy.
"""

with llm_config:
    product_finder = ConversableAgent(
        name="finder_agent",
        system_message=finder_message,
        description="Call this agent if you want to find new products."
    )

    purchase_advisor = ConversableAgent(
        name="advisor_agent",
        system_message=advisor_message,
        description="Call this agent if you need advice or seek more information about a product."
    )

    checkout_assistant = ConversableAgent(
        name="checkout_agent",
        system_message=checkout_message,
        description="Call this agent for a brief checkout summary including product, price, shipping methods."
    )

customer = ConversableAgent(
    name="customer_agent",
    system_message=customer_message,
    llm_config=False,
    human_input_mode='ALWAYS'
)


# Create the pattern

In [16]:
pattern = AutoPattern(
    initial_agent=product_finder,         # Start with the customer
    agents=[product_finder, purchase_advisor, checkout_assistant],           # All agents in the group chat
    user_agent=customer,                            # Provide our human-in-the-loop agent
    group_manager_args={"llm_config": llm_config}  # Config for group manager
)

# Initialize the group chat

In [18]:
initial_prompt = """I'm looking for a Laptop suitable for training deep learning models."""

result, context_variables, last_agent = initiate_group_chat(
    pattern=pattern,
    messages=initial_prompt,                     # Initial request with transactions
)

customer_agent (to chat_manager):

I'm looking for a Laptop suitable for training deep learning models.

--------------------------------------------------------------------------------

Next speaker: finder_agent


>>>>>>>> USING AUTO REPLY...
finder_agent (to chat_manager):

I recommend the **ASUS ROG Zephyrus G15**. It features powerful NVIDIA GeForce RTX 3060 or 3070 graphics and a high-performance AMD Ryzen processor, making it an excellent choice for deep learning tasks.

--------------------------------------------------------------------------------

Next speaker: advisor_agent


>>>>>>>> USING AUTO REPLY...
advisor_agent (to chat_manager):

**ASUS ROG Zephyrus G15**

**Pros:**

1. **Powerful Graphics:** Equipped with NVIDIA GeForce RTX 3060 or 3070, which supports CUDA cores essential for deep learning applications.
2. **High-Performance Processor:** Features AMD Ryzen 7 or 9 options, providing excellent multi-threading performance for training models.
3. **High Refresh Rate D

Replying as customer_agent. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Can you suggest another one from Dell brand.


customer_agent (to chat_manager):

Can you suggest another one from Dell brand.

--------------------------------------------------------------------------------

Next speaker: advisor_agent


>>>>>>>> USING AUTO REPLY...
advisor_agent (to chat_manager):

**Dell XPS 15**

**Pros:**

1. **High-Performance Specs:** Available with up to NVIDIA GeForce RTX 3050 or 3060 graphics, which can handle deep learning tasks.
2. **Powerful CPU Options:** Features Intel Core i7 or i9 processors, providing excellent computational power for model training.
3. **Stunning Display:** Options for a 4K OLED screen that offers vibrant colors, great for data visualization and presentations.
4. **Solid Build Quality:** Premium design with a sturdy chassis, making it durable for everyday use.
5. **Good Battery Life:** Typically better battery optimization compared to gaming laptops, which can be a plus for mobile use.

**Cons:**

1. **Performance vs. Gaming Laptops:** While it has decent specs, it may not perfo

Replying as customer_agent. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  looks good. I'll take this one


customer_agent (to chat_manager):

looks good. I'll take this one

--------------------------------------------------------------------------------

Next speaker: advisor_agent


>>>>>>>> USING AUTO REPLY...
advisor_agent (to chat_manager):

Great choice! The **Dell XPS 15** is a powerful and versatile laptop that should serve you well for training deep learning models and other tasks. Make sure to check for the specific configuration that fits your needs best—especially with respect to the GPU and CPU options.

If you have any other questions, need further assistance, or additional recommendations in the future, feel free to ask. Happy purchasing!

--------------------------------------------------------------------------------

Next speaker: checkout_agent


>>>>>>>> USING AUTO REPLY...
checkout_agent (to chat_manager):

### Checkout Summary

**Product:** Dell XPS 15  
**Estimated Price:** $1,600 - $2,500 (depending on configuration)  
**Shipping Method:** Standard shipping (estimate

Replying as customer_agent. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (7ffe8c49-8d6b-4bab-a556-b0b14c16912f): User requested to end the conversation

>>>>>>>> TERMINATING RUN (eb68443d-5cdb-464e-9b6a-c40426606273): No reply generated
